In [1]:
%matplotlib inline


import pandas as pd
import scipy as stats
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

C:\Users\miste\Miniconda3\envs\tutorial\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
#Importing the three dataframes CSV files
stats2018 = pd.read_csv('Data/2017-2018stats.txt')
salaries2019 = pd.read_csv('Data/2018-2019salaries.txt')
stats2018advanced = pd.read_csv('Data/2017-2018advanced.txt')

In [4]:
stats2018.columns

Index([u'Rk', u'Player', u'Pos', u'Age', u'Tm', u'G', u'GS', u'MP', u'FG',
       u'FGA', u'FG%', u'3P', u'3PA', u'3P%', u'2P', u'2PA', u'2P%', u'eFG%',
       u'FT', u'FTA', u'FT%', u'ORB', u'DRB', u'TRB', u'AST', u'STL', u'BLK',
       u'TOV', u'PF', u'PS/G'],
      dtype='object')

In [109]:
#Checking to see which players played for multiple teams 
multiple_teams = stats2018['Name'].loc[stats2018['Tm'] == 'TOT']
multiple_teams.count()



    



59

In [110]:
#Counting up all of the times a person who has played for multiple teams shows up in the dataframe
count_multiple = sum(stats2018['Name'].isin(multiple_teams))
count_multiple

183

In [111]:
#Subtracting away the amount of times the TOT shows up from the total
count_multiple - len(multiple_teams)

124

In [112]:
#Dropping the duplicates caused by a player changing teams midway, keeping the TOT entry. The amount makes sense as 660 - 124 = 540
stats2018u = stats2018.drop_duplicates(subset = ['Name'])
stats2018u.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540 entries, 0 to 663
Data columns (total 17 columns):
Name    540 non-null object
Pos     540 non-null object
Tm      540 non-null object
G       540 non-null int64
GS      540 non-null int64
FG      540 non-null float64
FG%     537 non-null float64
3P      540 non-null float64
3P%     491 non-null float64
eFG%    537 non-null float64
FT%     501 non-null float64
ORB     540 non-null float64
TRB     540 non-null float64
AST     540 non-null float64
STL     540 non-null float64
BLK     540 non-null float64
PS/G    540 non-null float64
dtypes: float64(12), int64(2), object(3)
memory usage: 75.9+ KB


In [113]:
#Splitting the weird Player column, and dropping all unneccessary columns
salaries2019[['Name','string']] = salaries2019['Player'].str.split('\\', expand = True)
dropped2019 = salaries2019.drop(['Player','2019-20','2020-21','2021-22','2022-23','2023-24', 'Signed Using','string','Guaranteed'],1)
dropped2019.head()

,﻿Rk,Tm,2018-19,Name
0,1,GSW,$37457154,Stephen Curry
1,2,LAL,$35654150,LeBron James
2,3,LAL,$35654150,LeBron James
3,4,HOU,$35654150,Chris Paul
4,5,OKC,$35350000,Russell Westbrook


In [114]:
#Counting to see how many nonunique entries there are. There are 450 unique entries in the name column and 2 that show up twice
len(dropped2019['Name']) - dropped2019['Name'].nunique()


2

In [115]:
#Counting to see how many entries there are
dropped2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452 entries, 0 to 451
Data columns (total 4 columns):
﻿Rk        452 non-null int64
Tm         452 non-null object
2018-19    452 non-null object
Name       452 non-null object
dtypes: int64(1), object(3)
memory usage: 17.7+ KB


In [116]:
#dropping duplicates that show up in the salary column for some reason. should be -2. This is correct
dropped2019u = dropped2019.drop_duplicates(subset = ['Name'])
dropped2019u.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 0 to 451
Data columns (total 4 columns):
﻿Rk        450 non-null int64
Tm         450 non-null object
2018-19    450 non-null object
Name       450 non-null object
dtypes: int64(1), object(3)
memory usage: 17.6+ KB


In [117]:
#Concatenating the two data frames together. Checking, we now only have 289 entries, because some players that played 
#last year do not have salaries for next year, and some who have salaries for next year did not play last year
data2018 = pd.merge(dropped2019u,stats2018u, how = 'inner', on = ['Name'])
data2018.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 0 to 377
Data columns (total 20 columns):
﻿Rk        378 non-null int64
Tm_x       378 non-null object
2018-19    378 non-null object
Name       378 non-null object
Pos        378 non-null object
Tm_y       378 non-null object
G          378 non-null int64
GS         378 non-null int64
FG         378 non-null float64
FG%        377 non-null float64
3P         378 non-null float64
3P%        352 non-null float64
eFG%       377 non-null float64
FT%        370 non-null float64
ORB        378 non-null float64
TRB        378 non-null float64
AST        378 non-null float64
STL        378 non-null float64
BLK        378 non-null float64
PS/G       378 non-null float64
dtypes: float64(12), int64(3), object(5)
memory usage: 62.0+ KB


In [118]:
#Taking the dollar sign out of the salary and casting it as an integer, so that I can order by it. Checking to see 
data2018['2018-19'] = data2018['2018-19'].str.replace('$', '').astype(int)

data2018.head()

,﻿Rk,Tm_x,2018-19,Name,Pos,Tm_y,G,GS,FG,FG%,3P,3P%,eFG%,FT%,ORB,TRB,AST,STL,BLK,PS/G
0,1,GSW,37457154,Stephen Curry,PG,GSW,51,51,8.4,0.495,4.2,0.423,0.618,0.921,0.7,5.1,6.1,1.6,0.2,26.4
1,2,LAL,35654150,LeBron James,PF,CLE,82,82,10.5,0.542,1.8,0.367,0.590,0.731,1.2,8.6,9.1,1.4,0.9,27.5
2,4,HOU,35654150,Chris Paul,PG,HOU,58,58,6.3,0.460,2.5,0.380,0.550,0.919,0.6,5.4,7.9,1.7,0.2,18.6
3,5,OKC,35350000,Russell Westbrook,PG,OKC,80,80,9.5,0.449,1.2,0.298,0.477,0.737,1.9,10.1,10.3,1.8,0.3,25.4
4,6,DET,31873932,Blake Griffin,PF,TOT,58,58,7.5,0.438,1.9,0.345,0.493,0.785,1.3,7.4,5.8,0.7,0.3,21.4


In [119]:
#Sorting by highest salary first and checking how it looks. Tm_y is an outdated team column, so I dropped it to remain constant
ordered2018 = data2018.sort_values(by = '2018-19', ascending = False)
singleteam = ordered2018.drop(['Tm_y'],1)



In [120]:
#renaming the Tm_x column to just be Tm for simplicity
data2018 = singleteam.rename(columns = {'Tm_x':'Tm'})
data2018.head()

,﻿Rk,Tm,2018-19,Name,Pos,G,GS,FG,FG%,3P,3P%,eFG%,FT%,ORB,TRB,AST,STL,BLK,PS/G
0,1,GSW,37457154,Stephen Curry,PG,51,51,8.4,0.495,4.2,0.423,0.618,0.921,0.7,5.1,6.1,1.6,0.2,26.4
2,4,HOU,35654150,Chris Paul,PG,58,58,6.3,0.460,2.5,0.380,0.550,0.919,0.6,5.4,7.9,1.7,0.2,18.6
1,2,LAL,35654150,LeBron James,PF,82,82,10.5,0.542,1.8,0.367,0.590,0.731,1.2,8.6,9.1,1.4,0.9,27.5
3,5,OKC,35350000,Russell Westbrook,PG,80,80,9.5,0.449,1.2,0.298,0.477,0.737,1.9,10.1,10.3,1.8,0.3,25.4
4,6,DET,31873932,Blake Griffin,PF,58,58,7.5,0.438,1.9,0.345,0.493,0.785,1.3,7.4,5.8,0.7,0.3,21.4


In [121]:
#Checking to see the column names of the advanced stats data set
stats2018advanced.columns

Index([u'Rk', u'Player', u'Pos', u'Age', u'Tm', u'G', u'MP', u'PER', u'TS%',
       u'3PAr', u'FTr', u'ORB%', u'DRB%', u'TRB%', u'AST%', u'STL%', u'BLK%',
       u'TOV%', u'USG%', u'Unnamed: 19', u'OWS', u'DWS', u'WS', u'WS/48',
       u'Unnamed: 24', u'OBPM', u'DBPM', u'BPM', u'VORP'],
      dtype='object')

In [122]:
#Cleaning the advanced stats dataframe and extracting meaningful columns
stats2018advanced[['Name','string']] = stats2018advanced['Player'].str.split('\\', expand = True)
advanced2018 = stats2018advanced[['Name','Age', 'WS','PER','MP','TRB%','AST%','TOV%','OWS','DWS']].drop_duplicates(subset = ['Name'])
advanced2018.head()

,Name,Age,WS,PER,MP,TRB%,AST%,TOV%,OWS,DWS
0,Alex Abrines,24,2.2,9.0,1134,5.6,3.4,7.4,1.3,1.0
1,Quincy Acy,27,1.0,8.2,1359,10.0,6.0,13.3,-0.1,1.1
2,Steven Adams,24,9.7,20.6,2487,15.3,5.5,13.2,6.7,3.0
3,Bam Adebayo,20,4.2,15.7,1368,15.6,11.0,13.6,2.3,1.9
4,Arron Afflalo,32,0.1,5.8,682,5.3,6.2,10.8,-0.1,0.2


In [123]:
#Merging with the previous data set that had the rows I desired so that this one also has only those rows
advancedclean = pd.merge(advanced2018,data2018[['Name','Tm']], how = 'inner', on = ['Name'])
advancedclean.head()

,Name,Age,WS,PER,MP,TRB%,AST%,TOV%,OWS,DWS,Tm
0,Alex Abrines,24,2.2,9.0,1134,5.6,3.4,7.4,1.3,1.0,OKC
1,Steven Adams,24,9.7,20.6,2487,15.3,5.5,13.2,6.7,3.0,OKC
2,Bam Adebayo,20,4.2,15.7,1368,15.6,11.0,13.6,2.3,1.9,MIA
3,Cole Aldrich,29,0.0,6.0,49,17.7,8.2,5.4,-0.1,0.1,MIN
4,LaMarcus Aldridge,32,10.9,25.0,2509,14.0,11.3,6.8,7.4,3.5,SAS
